# Calculate Cosine Similarity Score (KSA, Linked In)

In [49]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:

JD = pd.read_csv('D:\\study\\Gmu\\Sem-4\\690\\Data\\JD\\Job_Des.csv')


KSA = pd.read_excel('D:\\study\\Gmu\\Sem-4\\690\\Data\\KSA.xlsx')

In [51]:
JD

,Date,Company,Title,Location,Description,Level,Type,Function,Industry,Link
0,3/2/2023,MCNC,Cybersecurity Analyst,"Durham, NC",Company Overview MCNC is a North Carolina non...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting,https://www.linkedin.com/jobs/view/cybersecuri...
1,3/8/2023,MCNC,Associate Cybersecurity Analyst,"Durham, NC",Company OverviewMCNC is a North Carol...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/associate-c...
2,2/23/2023,CuraLinc Healthcare,Security Analyst,United States,Company OverviewMCNC is a North Carol...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/security-an...
3,3/18/2023,VForce Infotech,Cyber Security Analyst,"Edison, NJ",CuraLinc Healthcare is seeking a tale...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/cyber-secur...
4,3/20/2023,Irvine Technology Corporation,Security Operations Center Analyst,"Houston, TX",CuraLinc Healthcare is seeking a tale...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/security-op...
...,...,...,...,...,...,...,...,...,...,...
145,2/22/2023,Qualis Corporation,Cyber Security Analyst,"Albuquerque, NM","Long Term Contract, Atlanta/Raleigh C...",Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cyber-secur...
146,3/15/2023,Acuity Brands,Cybersecurity Intern,"Conyers, GA","Long Term Contract, Atlanta/Raleigh C...",Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cybersecuri...
147,3/20/2023,Defense Information Systems Agency,IT SPECIALIST (NETWORK),"Scott AFB, IL","Acuity Brands, Inc. (NYSE: AYI) is a ...",Internship,Internship,Information Technology,Electric Lighting Equipment Manufac...,https://www.linkedin.com/jobs/view/it-speciali...
148,10/8/2022,Center for Internet Security,Cyber Incident Response Team Intern - Remote,United States,Summary This position is being recruited u...,NaN,Internship,Other,Computer and Network Security,https://www.linkedin.com/jobs/view/cyber-incid...


In [52]:
KSA

,OPM Code,NICE Specialty Area,NICE Specialty Area Description,Work Role,Work Role Description,KSA ID,KSA Description
0,511,Cybersecurity Defense Analysis (CDA),Uses defensive measures and information collec...,Cyber Defense Analyst,Uses data collected from a variety of cyber de...,A0010,Ability to analyze malware.
1,461,Systems Analysis (ANA),Studies an organization's current computer sys...,Systems Security Analyst,Responsible for the analysis and development o...,A0015,Ability to conduct vulnerability scans and rec...
2,511,Cybersecurity Defense Analysis (CDA),Uses defensive measures and information collec...,Cyber Defense Analyst,Uses data collected from a variety of cyber de...,A0066,Ability to accurately and completely source al...
3,531,Incident Response (CIR),Responds to crises or urgent situations within...,Cyber Defense Incident Responder,"Investigates, analyzes, and responds to cyber ...",A0121,Ability to design incident response for cloud ...
4,461,Systems Analysis (ANA),Studies an organization's current computer sys...,Systems Security Analyst,Responsible for the analysis and development o...,A0123,Ability to apply cybersecurity and privacy pri...
...,...,...,...,...,...,...,...
124,511,Cybersecurity Defense Analysis (CDA),Uses defensive measures and information collec...,Cyber Defense Analyst,Uses data collected from a variety of cyber de...,S0169,Skill in conducting trend analysis.
125,531,Incident Response (CIR),Responds to crises or urgent situations within...,Cyber Defense Incident Responder,"Investigates, analyzes, and responds to cyber ...",S0173,Skill in using security event correlation tools.
126,531,Incident Response (CIR),Responds to crises or urgent situations within...,Cyber Defense Incident Responder,"Investigates, analyzes, and responds to cyber ...",S0365,Skill to design incident response for cloud se...
127,461,Systems Analysis (ANA),Studies an organization's current computer sys...,Systems Security Analyst,Responsible for the analysis and development o...,S0367,Skill to apply cybersecurity and privacy princ...


In [54]:

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', str(text).lower())
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stem the words
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text



JD['Description'] = JD['Description'].apply(preprocess_text)
KSA['KSA Description'] = KSA['KSA Description'].apply(preprocess_text)

In [55]:
JD['Description']

0      compani overview mcnc north carolina nonprofit...
1      compani overviewmcnc north carolina nonprofit ...
2      compani overviewmcnc north carolina nonprofit ...
3      curalinc healthcar seek talent secur analyst j...
4      curalinc healthcar seek talent secur analyst j...
                             ...                        
145    long term contract atlantaraleigh candid requi...
146    long term contract atlantaraleigh candid requi...
147    acuiti brand inc nyse ayi marketlead industri ...
148    summari posit recruit 10 usc 1599f cyber excep...
149    overviewth primari purpos posit gain experi pr...
Name: Description, Length: 150, dtype: object

In [56]:
KSA['KSA Description']

0                                     abil analyz malwar
1      abil conduct vulner scan recogn vulner secur s...
2      abil accur complet sourc data use intellig ass...
3           abil design incid respons cloud servic model
4      abil appli cybersecur privaci principl organiz...
                             ...                        
124                          skill conduct trend analysi
125                    skill use secur event correl tool
126        skill design incid respons cloud servic model
127    skill appli cybersecur privaci principl organi...
128    skill use cyber defens servic provid report st...
Name: KSA Description, Length: 129, dtype: object

In [57]:

KSA_string = ' '.join(KSA['KSA Description'])


KSA_df = pd.DataFrame({'KSA': KSA_string}, index=[0])


KSA_df

,KSA
0,abil analyz malwar abil conduct vulner scan re...


In [58]:

concatenated_text = pd.concat([JD['Description'], KSA_df['KSA']], ignore_index=True)


concatenated_text

0      compani overview mcnc north carolina nonprofit...
1      compani overviewmcnc north carolina nonprofit ...
2      compani overviewmcnc north carolina nonprofit ...
3      curalinc healthcar seek talent secur analyst j...
4      curalinc healthcar seek talent secur analyst j...
                             ...                        
146    long term contract atlantaraleigh candid requi...
147    acuiti brand inc nyse ayi marketlead industri ...
148    summari posit recruit 10 usc 1599f cyber excep...
149    overviewth primari purpos posit gain experi pr...
150    abil analyz malwar abil conduct vulner scan re...
Length: 151, dtype: object

## Approach 1. use CountVectorizer

In [59]:

ctvectorizer = CountVectorizer()


ct_matrix = ctvectorizer.fit_transform(concatenated_text)


ct_matrix

<151x2576 sparse matrix of type '<class 'numpy.int64'>'
	with 37210 stored elements in Compressed Sparse Row format>

In [60]:

cosine_ct = cosine_similarity(ct_matrix[:-1], ct_matrix[-1])



JD['cos_KSA_countvect'] = cosine_ct[:,0]

In [61]:

JD['cos_KSA_countvect']=JD['cos_KSA_countvect']*100

In [62]:

print(JD['cos_KSA_countvect'].describe())

count    150.000000
mean      26.220364
std        7.301775
min        0.000000
25%       22.784938
50%       27.170998
75%       28.892173
max       41.324703
Name: cos_KSA_countvect, dtype: float64


## Approach 2. use TF-IDF vectorizer

In [63]:

tfvectorizer = TfidfVectorizer()


tfidf_matrix = tfvectorizer.fit_transform(concatenated_text)


cosine_tfidf = cosine_similarity(tfidf_matrix[:-1], tfidf_matrix[-1])



JD['cos_KSA_tfidf'] = cosine_tfidf[:,0]

In [64]:

JD['cos_KSA_tfidf']=JD['cos_KSA_tfidf']*100

In [65]:

print(JD['cos_KSA_tfidf'].describe())

count    150.000000
mean      14.108073
std        5.147235
min        0.000000
25%       12.446305
50%       12.939286
75%       16.175558
max       26.517917
Name: cos_KSA_tfidf, dtype: float64


In [66]:
JD

,Date,Company,Title,Location,Description,Level,Type,Function,Industry,Link,cos_KSA_countvect,cos_KSA_tfidf
0,3/2/2023,MCNC,Cybersecurity Analyst,"Durham, NC",compani overview mcnc north carolina nonprofit...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting,https://www.linkedin.com/jobs/view/cybersecuri...,28.142595,12.352897
1,3/8/2023,MCNC,Associate Cybersecurity Analyst,"Durham, NC",compani overviewmcnc north carolina nonprofit ...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/associate-c...,28.911305,12.280979
2,2/23/2023,CuraLinc Healthcare,Security Analyst,United States,compani overviewmcnc north carolina nonprofit ...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/security-an...,27.511362,11.907829
3,3/18/2023,VForce Infotech,Cyber Security Analyst,"Edison, NJ",curalinc healthcar seek talent secur analyst j...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/cyber-secur...,28.601465,12.556311
4,3/20/2023,Irvine Technology Corporation,Security Operations Center Analyst,"Houston, TX",curalinc healthcar seek talent secur analyst j...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/security-op...,28.601465,12.556311
...,...,...,...,...,...,...,...,...,...,...,...,...
145,2/22/2023,Qualis Corporation,Cyber Security Analyst,"Albuquerque, NM",long term contract atlantaraleigh candid requi...,Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cyber-secur...,14.163023,4.029252
146,3/15/2023,Acuity Brands,Cybersecurity Intern,"Conyers, GA",long term contract atlantaraleigh candid requi...,Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cybersecuri...,14.163023,4.029252
147,3/20/2023,Defense Information Systems Agency,IT SPECIALIST (NETWORK),"Scott AFB, IL",acuiti brand inc nyse ayi marketlead industri ...,Internship,Internship,Information Technology,Electric Lighting Equipment Manufac...,https://www.linkedin.com/jobs/view/it-speciali...,15.235382,6.310918
148,10/8/2022,Center for Internet Security,Cyber Incident Response Team Intern - Remote,United States,summari posit recruit 10 usc 1599f cyber excep...,NaN,Internship,Other,Computer and Network Security,https://www.linkedin.com/jobs/view/cyber-incid...,21.724797,11.874771


In [67]:
JD[(JD['cos_KSA_countvect'] > 10) ]

,Date,Company,Title,Location,Description,Level,Type,Function,Industry,Link,cos_KSA_countvect,cos_KSA_tfidf
0,3/2/2023,MCNC,Cybersecurity Analyst,"Durham, NC",compani overview mcnc north carolina nonprofit...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting,https://www.linkedin.com/jobs/view/cybersecuri...,28.142595,12.352897
1,3/8/2023,MCNC,Associate Cybersecurity Analyst,"Durham, NC",compani overviewmcnc north carolina nonprofit ...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/associate-c...,28.911305,12.280979
2,2/23/2023,CuraLinc Healthcare,Security Analyst,United States,compani overviewmcnc north carolina nonprofit ...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,https://www.linkedin.com/jobs/view/security-an...,27.511362,11.907829
3,3/18/2023,VForce Infotech,Cyber Security Analyst,"Edison, NJ",curalinc healthcar seek talent secur analyst j...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/cyber-secur...,28.601465,12.556311
4,3/20/2023,Irvine Technology Corporation,Security Operations Center Analyst,"Houston, TX",curalinc healthcar seek talent secur analyst j...,Mid-Senior level,Full-time,Information Technology,Wellness and Fitness Services ...,https://www.linkedin.com/jobs/view/security-op...,28.601465,12.556311
...,...,...,...,...,...,...,...,...,...,...,...,...
145,2/22/2023,Qualis Corporation,Cyber Security Analyst,"Albuquerque, NM",long term contract atlantaraleigh candid requi...,Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cyber-secur...,14.163023,4.029252
146,3/15/2023,Acuity Brands,Cybersecurity Intern,"Conyers, GA",long term contract atlantaraleigh candid requi...,Mid-Senior level,Contract,"Information Technology, Engineeri...",Banking and Financial Services ...,https://www.linkedin.com/jobs/view/cybersecuri...,14.163023,4.029252
147,3/20/2023,Defense Information Systems Agency,IT SPECIALIST (NETWORK),"Scott AFB, IL",acuiti brand inc nyse ayi marketlead industri ...,Internship,Internship,Information Technology,Electric Lighting Equipment Manufac...,https://www.linkedin.com/jobs/view/it-speciali...,15.235382,6.310918
148,10/8/2022,Center for Internet Security,Cyber Incident Response Team Intern - Remote,United States,summari posit recruit 10 usc 1599f cyber excep...,NaN,Internship,Other,Computer and Network Security,https://www.linkedin.com/jobs/view/cyber-incid...,21.724797,11.874771


In [68]:
J1 = JD['Location'].str.split(",", expand = True)

In [69]:
J1.columns=["place","State"]

In [70]:
JD=pd.concat([JD,J1],axis=1)

In [71]:
JD=JD.drop(columns=['Location'])

In [72]:
JD.to_csv("D:\\study\\Gmu\\Sem-4\\690\\Data\\JD_Li.csv")